In [8]:
# Импорт библиотек
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import re

In [43]:
# Подгружаем файл с адресами
df = pd.read_excel('data.xlsx')

In [44]:
# Проходим по всем адресам в YandexMap через webdriver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

driver.get("https://yandex.ru/maps/")
html = driver.page_source
pattern = r'input__control _bold" id="(.*)" type="text"'
id_ = re.findall(pattern, html)[0]
time.sleep(3)

for x, y in zip(df['Adress'], df.index):
    
    element = driver.find_element(By.ID, id_)
    element.send_keys(x + '\n')
    time.sleep(0.5)
    
    html = driver.page_source
    geo_pattern = r'toponym-card-title-view__coords-badge">(.*)<span class="toponym-card-title-view__coords-copy-icon">'
    try: 
        geo_cor = re.findall(geo_pattern, html)[0]
    except:
        geo_cor = 'ND'
        
    df.loc[y, 'geo'] = geo_cor
        
    element.send_keys(Keys.CONTROL + "a")
    element.send_keys(Keys.BACK_SPACE)

In [ ]:
# Выгружаем файл с адресами и координатами
df.to_excel('geo_data.xlsx')